In [15]:
import sys
# Redirect stdout to a file for regular output
# sys.stdout = open('/home/ak-research-01/BP/Backpack-LM-debias/output/ranker_gpt2_Eval.out', 'w')
# sys.stderr = open('/home/ak-research-01/BP/Backpack-LM-debias/error/ranker_gpt2_Eval.rr', 'w')
import csv
import os
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import numpy as np
import json
import gzip
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments,DataCollatorWithPadding

# from transformers.trainer_for_eval_ranking import Trainer 

from sklearn.model_selection import train_test_split
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# import wandb



In [19]:
from modeling_backpack_gpt2 import BackpackGPT2LMHeadModel
from configuration_backpack_gpt2 import BackpackGPT2Config


class BackPackWithSigmoid(BackpackGPT2LMHeadModel):
    def __init__(self, bp_model):
        super(BackpackGPT2LMHeadModel, self).__init__(bp_model.config)
        self.backpack = bp_model.backpack  # Copy the transformer part of the model
        self.lm_head = bp_model.lm_head  # Copy the language modeling head
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation

    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        # Get the outputs from the base GPT-2 model
        outputs = super().forward(input_ids=input_ids)#, attention_mask=attention_mask, **kwargs)
        
        # Apply sigmoid activation to the logits
        logits = outputs.logits
        logits = self.sigmoid(logits)
        
        # Replace the logits in the output with the sigmoid-applied logits
        outputs.logits = logits
        
        return outputs


# Load JSON file
with open('/home/ak-research-01/BP/Backpack-LM-debias/config.json', 'r') as file:
    conf = json.load(file)

config = BackpackGPT2Config(**conf) 
model = BackpackGPT2LMHeadModel(config)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

checkpoint_path = '/home/ak-research-01/BP/Backpack-LM-debias/pytorch_model.bin'
state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Filter the external state_dict to retain only the keys present in the model's state_dict
filtered_state_dict = {k: v for k, v in state_dict.items() if k in model.state_dict()} 

model.load_state_dict(state_dict=filtered_state_dict)

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Add special tokens
special_tokens_dict = {'eos_token': '<EOS>', 'pad_token': '<PAD>'}
tokenizer.add_special_tokens(special_tokens_dict)

model.lm_head = nn.Linear(in_features=768, out_features=1, bias=False)

# Wrap it with the sigmoid
BP_model_with_sigmoid = BackPackWithSigmoid(model)

BP_model_with_sigmoid.load_state_dict(torch.load('/home/ak-research-01/BP/Backpack-LM-debias/models/rankers/BP_ranker/BP_ranker_with_sigmoid.pth'))
BP_model_with_sigmoid.eval()

/tmp/ipykernel_940451/3498425009.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))
/tmp/ipykern

BackPackWithSigmoid(
  (backpack): BackpackGPT2Model(
    (gpt2_model): GPT2Model(
      (wte): Embedding(50264, 768)
      (wpe): Embedding(512, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2SdpaAttention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (sense_network): BackpackSenseNetwork(
      (dropout): Dropout(p=0.1, inplace=False)
      

In [17]:
# inputs = [
#     ("How do I talk about feelings shy person", "One of the hardest things to do when growing up is tell a girl your feelings. Unfortunately, it sometimes doesn't get any easier. While telling a girl your feelings is tough when you are shy, you should always remember that she can't read minds, and you'll have to make a move sooner or later."),
# ("How to talk about sensitive topics in relationship", "So just because you've never seen your girl cry doesn't mean she doesn't occasionally want to, especially if you poke fun at a topic she's particularly sensitive about."),
# ("what are the benefits of fossil fuels","Fossil fuels are energy resources that come from the remains of plants and animals. These remains are millions of years old. There are three fossil fuels: petroleum oil, natural gas, and coal. Fossil fuels, like coal, oil, and natural gas, provide the energy that powers our lifestyles and our economy. Fossil fuels power everything from the planes in the sky to the cars on the road. They heat our homes and light up the night. Theyre the bedrock we base our energy mix on."),
# ("#ffffff color code","Color hex is a easy to use tool to get the color codes information including color models (RGB,HSL,HSV and CMYK), css and html color codes."),
# ("why did qbe stock drop?","After today’s latest let down the group’s chief executive John Neal announced the CEO of the ANZ business would be leaving QBE “effective immediately”. In May 2016 the group appointed a new CEO to its struggling North America operations and the group took on a new chief financial officer and chief risk officer as recently as 2014."),
# ("#ffffff color code","CSS Codes; Color Preview; Color Schemes; Color Picker; #ffffff is being represented by the (255,255,255) RGB color. It means that the #ffffff hex color has a Red value of 255, a Green value of 255 and a Blue value of 255. In adition, #ffffff hex color can also be represented as HSL color. The HSL color that represent the #ffffff has a Heu: 0.00, saturation: 0.00 and a lightness value of ffffff is 1.00. The #ffffff hex color has one more representation that worth mention - the 4 color CMYK, also known as the process color."),
# ("symptoms of hypercalcemia of malignancy","If you have cancer, your doctor will discuss treatment options with you to help you determine the best ways to treat hypercalcemia. You might be able to get relief from symptoms through intravenous fluids and medications like bisphosphonates."),
# ("symptoms of hypercalcemia of malignancy","Because of the clinical findings that are seen in hypercalcemia, the symptoms are often described as moans, stones, groans, and bones. Some of the symptoms include the following: Moans (gastrointestinal conditions) Constipation; Nausea; Decreased appetite; Abdominal pain; Peptic ulcer disease; Stones (kidney-related conditions) Kidney stones; Flank pain"),
# ("symptoms of hypercalcemia of malignancy","Hypercalcaemia, also spelled hypercalcemia, is a high calcium (Ca2+) level in the blood serum. The normal range is 2.1–2.6 mmol/L (8.8–10.7 mg/dL, 4.3–5.2 mEq/L) with levels greater than 2.6 mmol/L defined as hypercalcemia. Those with a mild increase that has developed slowly typically have no symptoms."),
# # ("symptoms of hypercalcemia of malignancy","Complicating diagnosis, the same symptoms common to advanced cancer are also seen in hypercalcemia of malignancy: nausea, constipation, anorexia, malaise and fatigue. [acphospitalist.org] Patients with mild hypercalcemia (serum calcium 10.5-12 mg/dL) can present with constipation, fatigue, and depression.")و
# ("gayla name meaning","In English, the name Gayla means-festive party.The name Gayla originated as an English name. The name Gayla is most often used as a girl name or female name. Meaning of Gayla. English name. In English, the name Gayla means-festive party.The name Gayla originated as an English name. The name Gayla is most often used as a girl name or female name. English Name Meaning-festive party."),
# ("gayla name meaning","Gayla /gay-la/ [2 sylls.] as a name for girls is of Hebrew and English derivation, and the name Gayla means father of exaltation. Gayla is an alternate spelling of Gail (Hebrew): short variant of Abigail. Gayla is also a derivative of Gayle (English). Kreatif forms: Gaella, Gayma, Gayta. (2000 U.S. Census). Displayed below is the birth name popularity of Gayla for girls. Gayla was first listed in 1940-1949 and reached its highest position of #493 in the U.S. during 1950-1959, but is not found in the list currently."),
# ("gayla name meaning","English Meaning: The name Gayla is an English baby name. In English the meaning of the name Gayla is: Festive party. American Meaning: The name Gayla is an American baby name. In American the meaning of the name Gayla is: Festive party. SoulUrge Number: 2. Expression Number: 1. People with this name have a deep inner desire for love and companionship, and want to work with others to achieve peace and harmony."),
# ("gayla name meaning","The name Gayla has the following meaning: My father's joy. An elaborated form of Gail, in turn a variant of Abigail. Also a variant of the French name Gala, meaning merry maker. The following names are baby names related to Gayla and are suitable names for brothers and sisters of Gayla: Boys-Amon, Areli, Asher, Elzie, Enoch, Jadon, Javon, Joel, Raphael, Girls-Anais, Derora, Diza, Mikaila, Ona, Yadira, Both genders-Jesse."),
# ("gayla name meaning","As a baby girl n "Gayla /gay-la/ [2 sylls.] as a name for girls is of Hebrew and English derivation, and the name Gayla means father of exaltation. Gayla is an alternate spelling of Gail (Hebrew): short variant of Abigail. Gayla is also a derivative of Gayle (English)."),
inputs = [("Gayla name meaning","As a baby girl name, Gayla is currently not a popular baby name in the USA. The following chart shows the popularity of the name Gayla in the USA, over the past 100 years. USA baby name statistics. You'll need the Adobe Flash Player and a browser with JavaScript enabled to see all baby name popularity charts. The following names are baby names related to Gayla and are suitable names for brothers and sisters of Gayla: Boys-Amon, Areli, Asher, Elzie, Enoch, Jadon, Javon, Joel, Raphael, Girls-Anais, Derora, Diza, Mikaila, Ona, Yadira, Both genders-Jesse."),
          ("Gayla name meaning", "Gayla /gay-la/ [2 sylls.] as a name for girls is of Hebrew and English derivation, and the name Gayla means father of exaltation. Gayla is an alternate spelling of Gail (Hebrew): short variant of Abigail. Gayla is also a derivative of Gayle (English)."),
          ("Gayla name meaning","In English, the name Gayla means 'festive party.' The name Gayla originated as an English name. The name Gayla is most often used as a girl name or female name. The meaning of Gayla: English name. In English, the name Gayla means 'festive party.' The name Gayla originated as an English name. The name Gayla is most often used as a girl name or female name. English Name Meaning: 'festive party.'"),
          ("Gayla name meaning","English Meaning: The name Gayla is an English baby name. In English, the meaning of the name Gayla is: Festive party. American Meaning: The name Gayla is an American baby name. In American, the meaning of the name Gayla is: Festive party. SoulUrge Number: 2. Expression Number: 1. People with this name have a deep inner desire for love and companionship, and want to work with others to achieve peace and harmony."),
          ("Gayla name meaning","The name Gayla has the following meaning: 'My father's joy.' An elaborated form of Gail, in turn a variant of Abigail. Also a variant of the French name Gala, meaning merry maker. The following names are baby names related to Gayla and are suitable names for brothers and sisters of Gayla: Boys-Amon, Areli, Asher, Elzie, Enoch, Jadon, Javon, Joel, Raphael. Girls-Anais, Derora, Diza, Mikaila, Ona, Yadira.")]
threshold = 0.5

# Tokenize inputs and pass them through the model
for query, document in inputs:
    # Concatenate the query and document with a separator
    input_text = query + " <EOS> " + document
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    # Pass through the model
    with torch.no_grad():
        outputs = BP_model_with_sigmoid(input_ids=input_ids)
    
    # Get the sigmoid-applied logits
    logits = outputs.logits.squeeze().cpu().numpy()
    print(logits.mean())
    print("###")
    relevance = "Relevant" if logits.mean() > threshold else "Non-Relevant"
    
    print(f"Input: {input_text}\nLogits: {logits}\nRelevance: {relevance}\n")



0.12706727
###
Input: Gayla name meaning <EOS> As a baby girl name, Gayla is currently not a popular baby name in the USA. The following chart shows the popularity of the name Gayla in the USA, over the past 100 years. USA baby name statistics. You'll need the Adobe Flash Player and a browser with JavaScript enabled to see all baby name popularity charts. The following names are baby names related to Gayla and are suitable names for brothers and sisters of Gayla: Boys-Amon, Areli, Asher, Elzie, Enoch, Jadon, Javon, Joel, Raphael, Girls-Anais, Derora, Diza, Mikaila, Ona, Yadira, Both genders-Jesse.
Logits: [3.8912341e-07 1.7393775e-04 8.9491135e-05 1.0000000e+00 7.7021876e-05
 3.0961892e-02 1.4401810e-01 6.5082538e-01 9.7171223e-01 9.8037708e-01
 9.4343448e-01 9.3345988e-01 5.9022957e-01 6.1445677e-01 9.9995971e-01
 9.9992454e-01 6.8361503e-01 6.6271907e-01 6.4270651e-01 8.1299788e-01
 7.2400969e-01 6.6415477e-01 5.2031529e-01 5.3514183e-01 6.2958640e-01
 7.3421180e-01 5.8708733e-01 1.8

In [20]:
import torch




threshold = 0.5  

def score_documents(query, documents):
    scores = []

    for document in documents:
        # Concatenate the query and document with a separator
        input_text = query + " <EOS> " + document
        input_ids = tokenizer.encode(input_text, return_tensors='pt')

        with torch.no_grad():
            outputs = BP_model_with_sigmoid(input_ids=input_ids)

        logits = outputs.logits.squeeze().cpu().numpy()
        score = logits.mean()
        
        scores.append((score, document))

    return scores

def rank_documents(scores):
    
    ranked_docs = sorted(scores, key=lambda x: x[0], reverse=True)
    
    return ranked_docs


query = "Gayla name meaning"
documents = [
    "As a baby girl name, Gayla is currently not a popular baby name in the USA. The following chart shows the popularity of the name Gayla in the USA, over the past 100 years. USA baby name statistics. You'll need the Adobe Flash Player and a browser with JavaScript enabled to see all baby name popularity charts. The following names are baby names related to Gayla and are suitable names for brothers and sisters of Gayla: Boys-Amon, Areli, Asher, Elzie, Enoch, Jadon, Javon, Joel, Raphael, Girls-Anais, Derora, Diza, Mikaila, Ona, Yadira, Both genders-Jesse.",
    "Gayla /gay-la/ [2 sylls.] as a name for girls is of Hebrew and English derivation, and the name Gayla means father of exaltation. Gayla is an alternate spelling of Gail (Hebrew): short variant of Abigail. Gayla is also a derivative of Gayle (English).",
    "In English, the name Gayla means 'festive party.' The name Gayla originated as an English name. The name Gayla is most often used as a girl name or female name. The meaning of Gayla: English name. In English, the name Gayla means 'festive party.' The name Gayla originated as an English name. The name Gayla is most often used as a girl name or female name. English Name Meaning: 'festive party.'",
    "English Meaning: The name Gayla is an English baby name. In English, the meaning of the name Gayla is: Festive party. American Meaning: The name Gayla is an American baby name. In American, the meaning of the name Gayla is: Festive party. SoulUrge Number: 2. Expression Number: 1. People with this name have a deep inner desire for love and companionship, and want to work with others to achieve peace and harmony.",
    "The name Gayla has the following meaning: 'My father's joy.' An elaborated form of Gail, in turn a variant of Abigail. Also a variant of the French name Gala, meaning merry maker. The following names are baby names related to Gayla and are suitable names for brothers and sisters of Gayla: Boys-Amon, Areli, Asher, Elzie, Enoch, Jadon, Javon, Joel, Raphael. Girls-Anais, Derora, Diza, Mikaila, Ona, Yadira."
]

# Score the documents
scores = score_documents(query, documents)

# Rank the documents
ranked_docs = rank_documents(scores)

# Print the ranked documents
for rank, (score, doc) in enumerate(ranked_docs, 1):
    #relevance = "Relevant" if score > threshold else "Non-Relevant"
    print(f"Rank {rank} | Score: {score} ")
    print(f"Document: {doc}\n")


Rank 1 | Score: 0.9420360326766968 
Document: English Meaning: The name Gayla is an English baby name. In English, the meaning of the name Gayla is: Festive party. American Meaning: The name Gayla is an American baby name. In American, the meaning of the name Gayla is: Festive party. SoulUrge Number: 2. Expression Number: 1. People with this name have a deep inner desire for love and companionship, and want to work with others to achieve peace and harmony.

Rank 2 | Score: 0.8835301995277405 
Document: In English, the name Gayla means 'festive party.' The name Gayla originated as an English name. The name Gayla is most often used as a girl name or female name. The meaning of Gayla: English name. In English, the name Gayla means 'festive party.' The name Gayla originated as an English name. The name Gayla is most often used as a girl name or female name. English Name Meaning: 'festive party.'

Rank 3 | Score: 0.8323329091072083 
Document: The name Gayla has the following meaning: 'My fa